![Screenshot 2024-11-25 at 7 12 58 PM](https://github.com/user-attachments/assets/48dd9de1-b4d2-4318-8f52-85ec209d8ebc)

# Install Necessary Libraries

In [ ]:
%pip install pygithub langchain langchain-community openai tiktoken pinecone-client langchain_pinecone sentence-transformers google-genai

In [16]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from langchain_pinecone import PineconeVectorStore
from langchain.embeddings import OpenAIEmbeddings
from langchain_community.embeddings import HuggingFaceEmbeddings
from google.colab import userdata
from pinecone import Pinecone
import os
import tempfile
from github import Github, Repository
from git import Repo
from openai import OpenAI
from pathlib import Path
from langchain.schema import Document
from pinecone import Pinecone

ModuleNotFoundError: No module named 'sentence_transformers'

# Clone a GitHub Repo locally

In [3]:
github_repo = "https://github.com/AbdulRahmanGit/DevDose"
namespace=github_repo.split("/")[-1]

'DevDose'

In [4]:
def clone_repository(repo_url):
    """Clones a GitHub repository to a temporary directory.

    Args:
        repo_url: The URL of the GitHub repository.

    Returns:
        The path to the cloned repository.
    """

    repo_name = github_repo.split("/")[-1]
    repo_path = f"/content/{repo_name}"
    Repo.clone_from(repo_url, str(repo_path))
    return repo_path

In [5]:
path = clone_repository(github_repo)

# Define which types of files to parse and which files / folders to ignore

In [6]:
SUPPORTED_EXTENSIONS = {'.py', '.js', '.tsx', '.jsx', '.ipynb', '.java',
                         '.cpp', '.ts', '.go', '.rs', '.vue', '.swift', '.c', '.h'}

IGNORED_DIRS = {'node_modules', 'venv', 'env', 'dist', 'build', '.git',
                '__pycache__', '.next', '.vscode', 'vendor'}

In [8]:
def get_file_content(file_path, repo_path):
    """
    Get content of a single file.

    Args:
        file_path (str): Path to the file

    Returns:
        Optional[Dict[str, str]]: Dictionary with file name and content
    """
    try:
        with open(file_path, "r", encoding="utf-8") as f:
            content = f.read()

        rel_path = os.path.relpath(file_path, repo_path)

        return {
            "name": rel_path,
            "content": content
        }

    except Exception as e:
        print(f"Error processing file {file_path}: {str(e)}")
        return None



def get_main_files_content(repo_path: str):
    """
    Get content of supported code files from the local repository.

    Args:
        repo_path: Path to the local repository

    Returns:
        List of dictionaries containing file names and contents
    """

    files_content = []

    try:

        for root, _, files in os.walk(repo_path):
            if any(ignored_dir in root for ignored_dir in IGNORED_DIRS):
                continue

            for file in files:
                file_path = os.path.join(root, file)
                if os.path.splitext(file)[1] in SUPPORTED_EXTENSIONS:
                    file_content = get_file_content(file_path, repo_path)

                    if file_content:
                        files_content.append(file_content)

    except Exception as e:
        print(e)

    return files_content


In [9]:
files_content = get_main_files_content(path)

In [10]:
files_content

[{'name': 'backend/database.py',
  'content': 'from sqlalchemy import create_engine, Column, String, Integer, inspect\nfrom sqlalchemy.orm import declarative_base, sessionmaker, Session\nfrom sqlalchemy.exc import SQLAlchemyError\nimport os\nfrom dotenv import load_dotenv\nfrom typing import Generator, Optional\n\n# Load environment variables\nload_dotenv()\n\n# Load database connection details from environment variables\nDB_HOST = os.getenv("POSTGRES_HOST")\nDB_NAME = os.getenv("POSTGRES_DB")\nDB_USER = os.getenv("POSTGRES_USER")\nDB_PASSWORD = os.getenv("POSTGRES_PASSWORD")\nDB_PORT = os.getenv("POSTGRES_PORT")\n\n# Define the database URL (Either from environment variable or construct manually)\nDATABASE_URL = os.getenv("POSTGRES_URL") or f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"\n\nif not DATABASE_URL:\n    raise ValueError("DATABASE_URL is not set. Ensure environment variables are correctly configured.")\n\n# Create the database engine\nengine = create_

# Embeddings

In [11]:
def get_huggingface_embeddings(text, model_name="sentence-transformers/all-mpnet-base-v2"):
    model = SentenceTransformer(model_name)
    return model.encode(text)

In [12]:
text = "I like coding"

embeddings = get_huggingface_embeddings(text)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [13]:
embeddings

array([-8.99214391e-03,  7.10570812e-02, -4.78132404e-02,  2.93951924e-03,
        1.87183879e-02,  6.25578165e-02, -7.40241334e-02, -1.79869793e-02,
       -2.33030953e-02,  2.52969563e-02,  1.87584516e-02,  3.26438658e-02,
        2.42187642e-02,  1.06377289e-01,  1.96239557e-02,  4.58664587e-03,
       -2.44689845e-02, -2.76926570e-02, -3.92995449e-03, -1.75862219e-02,
        7.31050968e-03,  3.00472043e-02, -3.48030999e-02, -1.51053397e-02,
        4.95744497e-03, -1.16299819e-02,  2.20015477e-02, -6.58035874e-02,
       -2.77131312e-02,  1.00825511e-01,  3.28472641e-04, -9.48085822e-03,
       -2.19271723e-02, -4.70387796e-03,  1.50509811e-06,  1.31359547e-02,
       -1.65854990e-02,  1.93589348e-02, -2.62506306e-02,  1.61743432e-04,
       -1.85249373e-02, -1.79329310e-02, -2.03854069e-02,  2.96985377e-02,
       -5.77551797e-02,  1.27474498e-02,  8.77798274e-02, -6.04865141e-02,
        9.92059615e-03, -2.96356127e-04,  1.08571432e-03, -1.75144020e-02,
       -3.35006081e-02, -

# Setting up Pinecone
**1. Create an account on [Pinecone.io](https://app.pinecone.io/)**

**2. Create a new index called "codebase-rag" and set the dimensions to 768. Leave the rest of the settings as they are.**

![Screenshot 2024-11-24 at 10 58 50 PM](https://github.com/user-attachments/assets/f5fda046-4087-432a-a8c2-86e061005238)



**3. Create an API Key for Pinecone**

![Screenshot 2024-11-24 at 10 44 37 PM](https://github.com/user-attachments/assets/e7feacc6-2bd1-472a-82e5-659f65624a88)


**4. Store your Pinecone API Key within Google Colab's secrets section, and then enable access to it (see the blue checkmark)**

![Screenshot 2024-11-24 at 10 45 25 PM](https://github.com/user-attachments/assets/eaf73083-0b5f-4d17-9e0c-eab84f91b0bc)



In [14]:
# Set the PINECONE_API_KEY as an environment variable
pinecone_api_key = userdata.get("PINECONE_API_KEY")
os.environ['PINECONE_API_KEY'] = pinecone_api_key

# Initialize Pinecone
pc = Pinecone(api_key=userdata.get("PINECONE_API_KEY"),)

# Connect to your Pinecone index
pinecone_index = pc.Index("codebase-rag")

In [15]:

vectorstore = PineconeVectorStore(index_name="codebase-rag", embedding=HuggingFaceEmbeddings())

<ipython-input-15-5982ffb8f713>:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  vectorstore = PineconeVectorStore(index_name="codebase-rag", embedding=HuggingFaceEmbeddings())
<ipython-input-15-5982ffb8f713>:1: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  vectorstore = PineconeVectorStore(index_name="codebase-rag", embedding=HuggingFaceEmbeddings())


In [16]:
# Insert the codebase embeddings into Pinecone

documents = []

for file in files_content:
    doc = Document(
        page_content=f"{file['name']}\n\n{file['content']}",
        metadata={"source": file['name']}
    )

    documents.append(doc)


vectorstore = PineconeVectorStore.from_documents(
    documents=documents,
    embedding=HuggingFaceEmbeddings(),
    index_name="codebase-rag",
    namespace=namespace
)

<ipython-input-16-49fec681b17e>:16: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embedding=HuggingFaceEmbeddings(),


In [17]:
documents

[Document(metadata={'source': 'backend/database.py', 'text': 'backend/database.py\n\nfrom sqlalchemy import create_engine, Column, String, Integer, inspect\nfrom sqlalchemy.orm import declarative_base, sessionmaker, Session\nfrom sqlalchemy.exc import SQLAlchemyError\nimport os\nfrom dotenv import load_dotenv\nfrom typing import Generator, Optional\n\n# Load environment variables\nload_dotenv()\n\n# Load database connection details from environment variables\nDB_HOST = os.getenv("POSTGRES_HOST")\nDB_NAME = os.getenv("POSTGRES_DB")\nDB_USER = os.getenv("POSTGRES_USER")\nDB_PASSWORD = os.getenv("POSTGRES_PASSWORD")\nDB_PORT = os.getenv("POSTGRES_PORT")\n\n# Define the database URL (Either from environment variable or construct manually)\nDATABASE_URL = os.getenv("POSTGRES_URL") or f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"\n\nif not DATABASE_URL:\n    raise ValueError("DATABASE_URL is not set. Ensure environment variables are correctly configured.")\n\n# Create

OpenRouter is a Platform for using multiple LLMs using single API Key

# Perform RAG
1. Get your OpenRouter API Key [here](https://openrouter.ai/settings/keys)

2. Paste your OpenRouter Key into your Google Colab secrets, and make sure to enable permissions for it

![Image](https://github.com/user-attachments/assets/bd64c5aa-952e-4a1e-9ac0-01d8fe93aaa1)


In [18]:
client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=userdata.get("OPENROUTER_API_KEY")
)

In [19]:
query = "How is the content generating"

In [20]:
raw_query_embedding = get_huggingface_embeddings(query)

In [21]:
raw_query_embedding

array([ 7.02322721e-02, -5.74920326e-02, -3.48516665e-02,  4.86997403e-02,
       -3.92402411e-02,  2.56798025e-02, -3.07044070e-02, -2.41490849e-03,
       -2.53554378e-02, -1.13149108e-02,  4.63070758e-02,  2.09454782e-02,
        1.86786093e-02,  6.77688792e-02,  2.18909308e-02, -3.37745547e-02,
        4.07367339e-03, -7.95187138e-04, -9.19094458e-02, -2.08136477e-02,
       -2.19556205e-02,  5.64904651e-04,  1.39239104e-02, -4.36566062e-02,
       -5.72504885e-02, -2.01363731e-02, -3.96462940e-02,  4.64733429e-02,
       -6.54492825e-02, -2.74089277e-02, -3.94473039e-02,  3.52100395e-02,
       -3.48867141e-02,  1.43951494e-02,  1.29384694e-06, -3.17980908e-02,
       -1.83093846e-02,  6.16373261e-03, -5.39679313e-03,  4.01272923e-02,
        5.71556352e-02,  6.01395406e-02, -1.21878358e-02,  4.44062352e-02,
       -3.65792820e-03,  3.68089899e-02,  3.64402533e-02,  7.29849329e-03,
        8.36807266e-02,  7.25067127e-03, -3.91384847e-02, -5.24650589e-02,
       -3.24478298e-02,  

In [22]:
top_matches = pinecone_index.query(vector=raw_query_embedding.tolist(),
                                   top_k=5,
                                   include_metadata=True,
                                   namespace=namespace)

In [23]:
top_matches

{'matches': [{'id': '30c444d8-543a-445f-955d-c65d4e16118d',
              'metadata': {'source': 'backend/generate.py',
                           'text': 'backend/generate.py\n'
                                   '\n'
                                   'import google.generativeai as genai\n'
                                   'import os\n'
                                   'from dotenv import load_dotenv\n'
                                   'import typing_extensions\n'
                                   'import json\n'
                                   '# Load environment variables\n'
                                   'load_dotenv()\n'
                                   'api_key = os.getenv("API_KEY")\n'
                                   '\n'
                                   '# Define the schema for the JSON response\n'
                                   'class '
                                   'ProgrammingTip(typing_extensions.TypedDict):\n'
                                

In [24]:
context = [item['metadata']['text'] for item in top_matches['matches']]

In [25]:
context

['backend/generate.py\n\nimport google.generativeai as genai\nimport os\nfrom dotenv import load_dotenv\nimport typing_extensions\nimport json\n# Load environment variables\nload_dotenv()\napi_key = os.getenv("API_KEY")\n\n# Define the schema for the JSON response\nclass ProgrammingTip(typing_extensions.TypedDict):\n    header_title: str\n    introduction_greeting: str\n    introduction_message: str\n    programming_tip_title: str\n    programming_tip_description: str\n    programming_tip_code: str\n    programming_tip_output: str\n    dsa_challenge_title: str\n    dsa_challenge_problem: str\n    dsa_challenge_solution_steps: str\n    dsa_challenge_code: str\n    dsa_problem_links:str\n    footer_message: str\n\n# Initialize the Gemini model\ngenai.configure(api_key=api_key)\nmodel = genai.GenerativeModel(\n    \'gemini-1.5-flash\',\n    generation_config={\n        "temperature": 1,\n  "top_p": 0.95,\n  "top_k": 64,\n  "max_output_tokens": 8192,\n  "response_mime_type": "application/j

In [26]:
augmented_query = "<CONTEXT>\n" + "\n\n-------\n\n".join(context) + "\n-------\n</CONTEXT>\n\n\n\nMY QUESTION:\n" + query

In [27]:
print(augmented_query)

<CONTEXT>
backend/generate.py

import google.generativeai as genai
import os
from dotenv import load_dotenv
import typing_extensions
import json
# Load environment variables
load_dotenv()
api_key = os.getenv("API_KEY")

# Define the schema for the JSON response
class ProgrammingTip(typing_extensions.TypedDict):
    header_title: str
    introduction_greeting: str
    introduction_message: str
    programming_tip_title: str
    programming_tip_description: str
    programming_tip_code: str
    programming_tip_output: str
    dsa_challenge_title: str
    dsa_challenge_problem: str
    dsa_challenge_solution_steps: str
    dsa_challenge_code: str
    dsa_problem_links:str
    footer_message: str

# Initialize the Gemini model
genai.configure(api_key=api_key)
model = genai.GenerativeModel(
    'gemini-1.5-flash',
    generation_config={
        "temperature": 1,
  "top_p": 0.95,
  "top_k": 64,
  "max_output_tokens": 8192,
  "response_mime_type": "application/json",
    }, system_instructio

In [40]:

from google import genai
from google.colab import userdata


GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
client = genai.Client(api_key=GOOGLE_API_KEY)

In [47]:
from google import genai

client = genai.Client(api_key=GOOGLE_API_KEY)
chat = client.chats.create(model="gemini-2.0-flash")

response = chat.send_message("How does Email is generated")
print(response.text)

for message in chat.get_history():
    print(f'role - {message.role}',end=": ")
    print(message.parts[0].text)

The generation of an email involves several steps and layers of technology working together. Here's a breakdown of how an email is generated from the user's perspective to its transmission across the internet:

**1. Composition (What the User Sees)**

*   **Email Client/Webmail:** The process begins with the user composing the email within an email client (like Outlook, Thunderbird, or Apple Mail) or through a webmail interface (like Gmail, Yahoo Mail, or Outlook.com).
*   **Input:** The user fills in the "To," "Cc," "Bcc," "Subject," and body fields of the email. They may also attach files.
*   **Formatting:** The email client allows the user to format the text using various fonts, sizes, colors, and styles. These formatting options are often converted into HTML or plain text.

**2. Encoding and Formatting (Behind the Scenes)**

*   **MIME (Multipurpose Internet Mail Extensions):** The email client uses MIME to format the email so that it can include various types of content, such as 

#DeepSeek

In [39]:


from openai import OpenAI

client = OpenAI(api_key="", base_url="https://api.deepseek.com")

response = client.chat.completions.create(
    model="deepseek-chat",
    messages=[
        {"role": "system", "content": "You are a helpful assistant"},
        {"role": "user", "content": "Hello"},
    ],
    stream=False
)

print(response.choices[0].message.content)


APIStatusError: Error code: 402 - {'error': {'message': 'Insufficient Balance', 'type': 'unknown_error', 'param': None, 'code': 'invalid_request_error'}}

Multi-Models

In [30]:
system_prompt = """You are a Senior Software Engineer,

Answer the question I have about the codebase based on the context provided. Always consider all of the context provided
to answer my question.
"""


llm_response = client.chat.completions.create(
    model="openai/gpt-4o",
    extra_body={
        "models": ["anthropic/claude-3.5-sonnet", "gryphe/mythomax-l2-13b"],
    },
    messages=[
        {
            "role": "user",
            "content": "What is the meaning of building this project?"
        }
    ]
)

response = llm_response.choices[0].message.content


AttributeError: 'Client' object has no attribute 'chat'

In [ ]:
print(response)

RAG using DeepSeek for Text Completions

In [54]:
def perform_rag(query, model="deepseek/deepseek-r1:free"):
    raw_query_embedding = get_huggingface_embeddings(query)

    top_matches = pinecone_index.query(vector=raw_query_embedding.tolist(), top_k=5, include_metadata=True, namespace="https://github.com/AbdulRahmanGit/DevDose")

    # Get the list of retrieved texts
    contexts = [item['metadata']['text'] for item in top_matches['matches']]

    augmented_query = "<CONTEXT>\n" + "\n\n-------\n\n".join(contexts[ : 10]) + "\n-------\n</CONTEXT>\n\n\n\nMY QUESTION:\n" + query

    # Modify the prompt below as need to improve the response quality
    system_prompt = f"""You are a Senior Software Engineer,

    Answer any questions I have about the codebase, based on the code provided. Always consider all of the context provided when forming a response.
    """

    llm_response = client.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": augmented_query}
        ]
    )

    if llm_response and llm_response.choices:
        return llm_response.choices[0].message.content
    else:
        print("Error: Language model API did not return any valid responses.")  # Print error for debugging
        return "No response from language model." # Or handle the error in another suitable way

In [5]:
def perform_rag(query, model="gemini-2.0-flash"):
    raw_query_embedding = get_huggingface_embeddings(query)

    top_matches = pinecone_index.query(vector=raw_query_embedding.tolist(), top_k=5, include_metadata=True, namespace="https://github.com/AbdulRahmanGit/DevDose")

    # Get the list of retrieved texts
    contexts = [item['metadata']['text'] for item in top_matches['matches']]

    augmented_query = "<CONTEXT>\n" + "\n\n-------\n\n".join(contexts[ : 10]) + "\n-------\n</CONTEXT>\n\n\n\nMY QUESTION:\n" + query

    # Modify the prompt below as need to improve the response quality
    system_prompt = f"""You are a Senior Software Engineer,

    Answer any questions I have about the codebase, based on the code provided. Always consider all of the context provided when forming a response.
    """

    llm_response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": augmented_query}
        ]
    )

    if llm_response and llm_response.choices:
        return llm_response.choices[0].message.content
    else:
        print("Error: Language model API did not return any valid responses.")  # Print error for debugging
        return "No response from language model." # Or handle the error in another suitable way

In [ ]:
response = perform_rag("Talk us through a time when you took ownership and figured out solutions to problems independently in this porject?")
from IPython.display import Markdown, display

print(response)
display(Markdown(response))

NameError: name 'get_huggingface_embeddings' is not defined

In [33]:
display(Markdown(response))

NameError: name 'Markdown' is not defined

In [34]:
response = perform_rag("What does this github repo do?", "anthropic/claude-3.7-sonnet:thinking")
from IPython.display import Markdown, display

display(Markdown(response))

AttributeError: 'Client' object has no attribute 'completions'